## Проект SQL

Задания

Посчитайте, сколько книг вышло после 1 января 2000 года;

Для каждой книги посчитайте количество обзоров и среднюю оценку;

Определите издательство, которое выпустило наибольшее число книг толще
50 страниц — так вы исключите из анализа брошюры;

Определите автора с самой высокой средней оценкой книг — учитывайте
только книги с 50 и более оценками;

Посчитайте среднее количество обзоров от пользователей, которые поставили
больше 48 оценок.

In [1]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa
# устанавливаем параметры
db_config = {
'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)
# формируем запрос и выводим данные
query = '''SELECT * FROM books LIMIT 5'''
get_sql_data(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Проверим содержимое всех таблиц.

In [2]:
query = '''SELECT * FROM books'''
books= get_sql_data(query)

In [3]:
query = '''SELECT * FROM publishers'''
publishers= get_sql_data(query)

In [4]:
query = '''SELECT * FROM author'''
author= get_sql_data(query)

In [5]:
query = '''SELECT * FROM ratings'''
ratings= get_sql_data(query)

In [6]:
query = '''SELECT * FROM reviews'''
reviews= get_sql_data(query)

In [7]:
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [8]:
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [9]:
author.head()

,index,author_id,author
0,0,1,A.S. Byatt
1,1,2,Aesop/Laura Harris/Laura Gibbs
2,2,3,Agatha Christie
3,3,4,Alan Brennert
4,4,5,Alan Moore/David Lloyd


In [10]:
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [11]:
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Задание 1.
Посчитайте, сколько книг вышло после 1 января 2000 года;

In [12]:
query = '''
SELECT COUNT(*) 
FROM books
WHERE publication_date > '2000-01-01'
'''
get_sql_data(query)

,count
0,819


После 1 января 2000 года вышло 819 книг

Задание 2. Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [13]:
query = '''
SELECT DISTINCT(b.book_id),
b.title AS book_title,
COUNT(DISTINCT(r.review_id)) AS review_count,
AVG(DISTINCT(ra.rating)) AS average_rating
FROM books b
LEFT JOIN reviews r ON b.book_id = r.book_id
LEFT JOIN ratings ra ON b.book_id = ra.book_id
GROUP BY b.book_id, b.title
ORDER BY average_rating DESC, review_count DESC
'''
get_sql_data(query)

,book_id,book_title,review_count,average_rating
0,17,A Dirty Job (Grim Reaper #1),4,5.0
1,444,Moneyball: The Art of Winning an Unfair Game,3,5.0
2,553,School's Out—Forever (Maximum Ride #2),3,5.0
3,20,A Fistful of Charms (The Hollows #4),2,5.0
4,55,A Woman of Substance (Emma Harte Saga #1),2,5.0
...,...,...,...,...
995,943,Tsubasa: RESERVoir CHRoNiCLE Vol. 1,1,2.5
996,772,The Kitchen God's Wife,3,2.0
997,316,His Excellency: George Washington,2,2.0
998,371,Junky,2,2.0


Больше всего отзывов у книг A Dirty Job (Grim Reaper #1),	Moneyball: The Art of Winning an Unfair Game и School's Out—Forever (Maximum Ride #2)	

Задание 3. 
Определите издательство, которое выпустило наибольшее число книг толще
50 страниц — так вы исключите из анализа брошюры;

In [14]:
query = '''
SELECT p.publisher_id,
p.publisher,
COUNT(b.book_id) AS book_count
FROM publishers p
JOIN books b ON p.publisher_id = b.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id, p.publisher
ORDER BY COUNT(b.book_id) DESC
LIMIT 1;
'''
get_sql_data(query)

,publisher_id,publisher,book_count
0,212,Penguin Books,42


издательство, которое выпустило наибольшее число книг толще 50 страниц - Penguin Books

Задание 4. Определите автора с самой высокой средней оценкой книг — учитывайте
только книги с 50 и более оценками;

In [15]:
query = '''
SELECT 
avr_aut.author,
AVG(avr_aut.average_rat) AS average_rating
FROM
(
SELECT
a.author,
AVG(r.rating) AS average_rat
from books b
JOIN authors a ON b.author_id = a.author_id
JOIN ratings r ON b.book_id = r.book_id
GROUP BY b.book_id,a.author_id
HAVING COUNT(r.rating_id) >= 50
) AS avr_aut
GROUP BY avr_aut.author
ORDER BY average_rating DESC
LIMIT 1;
'''
get_sql_data(query)

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.283844


Авторы с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré.

Задание 5. Посчитайте среднее количество обзоров от пользователей, которые поставили
больше 48 оценок.

In [16]:
query = '''
SELECT
AVG(subquery.cnt_reviews)
FROM
(SELECT
username,
COUNT(review_id) AS cnt_reviews
FROM
reviews
WHERE 
username IN (
            SELECT 
            username
            FROM 
            ratings
            GROUP BY 
            username
            HAVING 
            COUNT(rating_id) > 48)
            GROUP BY 
            username
            ) AS subquery
    '''
get_sql_data(query)

,avg
0,24.0


Количество обзоров от пользователей, которые поставили больше 48 оценок - 24.